In [1]:
!pip install klib sentencepiece transformers gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 58.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 114.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.3/140.3 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.8 MB/s eta 0:00:00

In [2]:
from transformers import  MT5ForConditionalGeneration, AutoTokenizer
import gradio as gr
import torch
import torch.nn as nn
from google.colab import drive
import warnings
warnings.filterwarnings("ignore")

In [3]:
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
model_repo = "google/mt5-small"

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_repo)
model =  MT5ForConditionalGeneration.from_pretrained(model_repo)

You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


In [6]:
SPECIAL_TOKEN_MAPPING = {
    'identify language': '<idf.lang>'
}
special_tokens_dict = {'additional_special_tokens': list(SPECIAL_TOKEN_MAPPING.values())}
tokenizer.add_special_tokens(special_tokens_dict)
model.resize_token_embeddings(len(tokenizer))

Embedding(250101, 512)

In [7]:
model.load_state_dict(torch.load("/content/drive/MyDrive/LS_NN_NLP/fine-tuned_mt5/Step-5500_checkpoint_lang_pred.pt"))

<All keys matched successfully>

In [14]:
def embed_str(text, tokenizer, seq_len):
  """ Tokenize, pad to max length and encode to ids
      Returns tensor with token ids """
  input_ids = tokenizer.encode(
      text=text,
      return_tensors = 'pt',
      padding = 'max_length',
      truncation = True,
      max_length = seq_len)

  return input_ids[0]

In [15]:
def embed_row(input_text, tokenizer, seq_inp_len):
  """Encode input and tagret texts from single row"""
  """Returns input and output token ids"""

  if input_text is None:
    return None

  input_token_ids = embed_str(
      input_text, tokenizer, seq_inp_len)

  return input_token_ids

In [16]:
lang_dict = {
    "ar" : "Arabian",
    "bg" : "Bulgarian",
    "de" : "German",
    "el" : "Greek",
    "en" : "English",
    "es" : "Spanish",
    "fr" : "French",
    "hi" : "Hindi",
    "ru" : "Russian",
    "sw" : "Swahili",
    "th" : "Thai",
    "tr" : "Turkish",
    "ur" : "Urdu",
    "vi" : "Vietnamese",
    "zh" : "Chinese (Simplified)"
             }

In [17]:
def response(text : str):
  text = "<idf.lang> " + text
  encoded_text = embed_row(text, tokenizer, 40)
  encoded_data = torch.cat([encoded_text.unsqueeze(0)])
  model_out = model.generate(encoded_data, num_beams=10, num_return_sequences=1, length_penalty = 1, no_repeat_ngram_size=2)[0]
  prediction = tokenizer.decode(model_out, skip_special_tokens=True)
  return  "The language of this text is " + lang_dict[prediction] + "."

In [18]:
demo = gr.Interface(fn=response, inputs='text', outputs='text')
demo.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>